In [1]:
!pip install transformers

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "iiiorg/piiranha-v1-detect-personal-information"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def mask_pii(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get the model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted labels
    predictions = torch.argmax(outputs.logits, dim=-1)

    # Convert token predictions to word predictions
    encoded_inputs = tokenizer.encode_plus(text, return_offsets_mapping=True, add_special_tokens=True)
    offset_mapping = encoded_inputs['offset_mapping']

    masked_text = list(text)

    for i, (start, end) in enumerate(offset_mapping):
        if start == end:  # Special token
            continue

        label = predictions[0][i].item()
        if label != model.config.label2id['O']:  # Non-O label
            pii_type = model.config.id2label[label]
            for j in range(start, end):
                masked_text[j] = f'[{pii_type}]'

    return ''.join(masked_text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/19.7k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [3]:
# Example usage
example_text = "My name is John Doe and I live at 123 Main St, New York. My phone number is 555-123-4567."
masked_example = mask_pii(example_text)
print("Example masked text:")
print(masked_example)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Example masked text:
My name is John Doe and I live at[I-BUILDINGNUM][I-BUILDINGNUM][I-BUILDINGNUM][I-BUILDINGNUM] Main St,[I-CITY][I-CITY][I-CITY][I-CITY][I-CITY][I-CITY][I-CITY][I-CITY][I-CITY]. My phone number is[I-TELEPHONENUM][I-TELEPHONENUM][I-TELEPHONENUM][I-TELEPHONENUM][I-TELEPHONENUM][I-TELEPHONENUM][I-TELEPHONENUM][I-TELEPHONENUM][I-TELEPHONENUM][I-TELEPHONENUM][I-TELEPHONENUM][I-TELEPHONENUM][I-TELEPHONENUM][I-TELEPHONENUM]
